In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx
from pathlib import Path

### Prepare map
This notebook prepares the files to produce the maps. A QGIS project file is provided in `map`.

In [2]:
# Inputs and outputs
resources_path = Path("../resources")
results_path = Path("../results")

map_path = Path("map")

In [3]:
# Prepare file containing zone geography
df_study_area = gpd.read_file(resources_path / "spatial/study_area.gpkg")

df_area = df_study_area[["geometry"]].dissolve()
df_area["name"] = "study_area"

df_zfe = gpd.read_file(resources_path / "spatial/zfe.gpkg")
df_zfe = df_zfe[["geometry"]]
df_zfe["name"] = "zfe"

df_metropole = pd.read_csv(resources_path / "spatial/metropole.csv")
df_metropole["municipality_id"] = df_metropole["municipality_id"].astype(str)
df_metropole = df_study_area[df_study_area["INSEE_COM"].isin(df_metropole["municipality_id"])][[
    "geometry"
]].dissolve()
df_metropole["name"] = "metropole"

df_spatial = pd.concat([
    df_area, df_zfe, df_metropole
])

df_spatial.to_file(map_path / "zones.gpkg")

In [5]:
# Prepare file providing network geometry
network = osmnx.load_graphml(results_path / "network/default.graphml")

df_network = osmnx.graph_to_gdfs(network, nodes = False, edges = True)
df_network = df_network.reset_index()[["highway", "geometry"]]
df_network = df_network.to_crs("EPSG:2154")
df_network["highway"] = df_network["highway"].apply(
    lambda x: x if type(x) == str else x[0]
)
df_network.to_file(map_path / "network.gpkg")

In [6]:
# Generate file containing depot information
df_counts = pd.read_parquet(results_path / "scenario_solutions/today_2024.vehicles.parquet")
df_counts = df_counts[["operator", "depot", "deliveries"]]
df_counts = df_counts.rename(columns = { "depot": "depot_id" })
df_counts["depot_id"] = df_counts["depot_id"].astype(str)
df_counts = df_counts.groupby(["depot_id", "operator"])["deliveries"].sum().reset_index()

df_spatial = gpd.read_file(results_path / "depots/located.gpkg")
df_spatial = df_spatial[["depot_id", "operator", "geometry"]]
df_spatial["depot_id"] = df_spatial["depot_id"].astype(str)
df_spatial = pd.merge(df_spatial, df_counts, on = ["depot_id", "operator"])

df_spatial = df_spatial.sort_values(by = "deliveries", ascending = False)
df_spatial["rank"] = np.arange(len(df_spatial))

df_spatial["operator"] = df_spatial["operator"].replace({
    "chronopost": "Chronopost",
    "ups": "UPS",
    "laposte": "La Poste",
    "dhl": "DHL",
    "gls": "GLS",
    "dpd": "DPD",
    "colisprive": "Colis privé"
})

df_spatial["count"] = df_spatial["deliveries"].apply("{:,d}".format)

df_spatial.to_file(map_path / "depots.gpkg")